## Section 1: Setup & Load Graph

In [ ]:
import os
import sys
import asyncio
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from typing import Dict, List, Set
from collections import defaultdict, Counter

import httpx
import redis.asyncio as redis
from neo4j import GraphDatabase
from neo4j.exceptions import ServiceUnavailable

# Try to import community detection algorithms
try:
    import community.community_louvain as community
except ImportError:
    print("⚠️  Install python-louvain: pip install python-louvain")
    community = None

plt.style.use('dark_background')
%matplotlib inline

print("✅ Libraries loaded")

In [ ]:
# Load environment and initialize connections (same as Notebook 01)
REDIS_URL = os.getenv('REDIS_URL', 'redis://redis:6379')
MEMGRAPH_URL = os.getenv('MEMGRAPH_URL', 'bolt://memgraph:7687')
BACKEND_API_URL = os.getenv('BACKEND_API_URL', 'http://code-graph-http:8000')

class GraphConnections:
    def __init__(self, redis_url: str, memgraph_url: str, api_url: str):
        self.redis_url = redis_url
        self.memgraph_url = memgraph_url
        self.api_url = api_url
        self.redis_client = None
        self.memgraph_driver = None
        self.http_client = None
    
    async def connect(self):
        try:
            self.redis_client = await redis.from_url(self.redis_url, decode_responses=True)
            await self.redis_client.ping()
            print("✅ Redis connected")
        except Exception as e:
            print(f"⚠️  Redis connection failed: {e}")
        
        try:
            self.memgraph_driver = GraphDatabase.driver(self.memgraph_url)
            with self.memgraph_driver.session() as session:
                session.run("RETURN 1").consume()
            print("✅ Memgraph connected")
        except:
            print("⚠️  Memgraph not available")
            self.memgraph_driver = None
        
        self.http_client = httpx.AsyncClient(base_url=self.api_url)
        print("✅ HTTP client ready")

connections = GraphConnections(REDIS_URL, MEMGRAPH_URL, BACKEND_API_URL)
loop = asyncio.new_event_loop()
asyncio.set_event_loop(loop)
loop.run_until_complete(connections.connect())

print("\n✅ Connections initialized")

In [ ]:
# Load graph data
async def get_all_nodes():
    try:
        response = await connections.http_client.get('/api/graph/nodes/search?limit=1000')
        data = response.json()
        return pd.DataFrame(data.get('results', []))
    except Exception as e:
        print(f"Error: {e}")
        return pd.DataFrame()

async def get_all_relationships():
    try:
        response = await connections.http_client.get('/api/graph/relationships?limit=5000')
        data = response.json()
        return data.get('results', [])
    except Exception as e:
        print(f"Error: {e}")
        return []

# Fetch data and build graph
nodes_df = loop.run_until_complete(get_all_nodes())
relationships = loop.run_until_complete(get_all_relationships())

# Build undirected graph for community detection (better modularity)
G = nx.Graph()

if not nodes_df.empty:
    for idx, row in nodes_df.iterrows():
        G.add_node(row['name'], type=row.get('type'), file=row.get('file'), language=row.get('language'))

for rel in relationships:
    if rel.get('source_name') in G and rel.get('target_name') in G:
        G.add_edge(rel['source_name'], rel['target_name'])

print(f"✅ Graph loaded: {G.number_of_nodes()} nodes, {G.number_of_edges()} edges")

## Section 2: Connected Components

Find structural components in the graph (strongly/weakly connected subgraphs).

In [ ]:
# Find connected components
components = list(nx.connected_components(G))
components_sorted = sorted(components, key=len, reverse=True)

print(f"🔍 Connected Components: {len(components)} found\n")

# Analyze top components
for i, component in enumerate(components_sorted[:10]):
    print(f"{i+1}. Component size: {len(component)} nodes")

# Show largest component stats
largest_cc = components_sorted[0]
G_largest = G.subgraph(largest_cc)

print(f"\n📊 Largest Connected Component:")
print(f"   Nodes: {G_largest.number_of_nodes()}")
print(f"   Edges: {G_largest.number_of_edges()}")
print(f"   Density: {nx.density(G_largest):.4f}")

## Section 3: Louvain Community Detection

Apply the Louvain algorithm to find optimal communities based on modularity.

In [ ]:
if community:
    print("⏳ Running Louvain algorithm...")
    partition = community.best_partition(G)
    modularity = community.modularity(partition, G)
    
    print(f"\n✅ Louvain Detection Complete")
    print(f"   Communities found: {len(set(partition.values()))}")
    print(f"   Modularity score: {modularity:.4f}")
    
    # Analyze community sizes
    community_sizes = Counter(partition.values())
    community_sizes = dict(sorted(community_sizes.items(), key=lambda x: x[1], reverse=True))
    
    print(f"\n📊 Community Sizes:")
    for comm_id, size in list(community_sizes.items())[:10]:
        print(f"   Community {comm_id}: {size} nodes")
else:
    print("⚠️  Install community detection: pip install python-louvain")
    partition = None

In [ ]:
if partition:
    # Show members of top 3 communities
    top_communities = sorted(partition.items(), key=lambda x: community_sizes.get(x[1], 0), reverse=True)[:30]
    
    for node, comm_id in top_communities:
        size = community_sizes.get(comm_id)
        if size > 5:  # Only show if community has >5 members
            print(f"Community {comm_id} ({size} members): {node}")
            break
    
    # Create community assignment mapping
    communities_by_id = defaultdict(list)
    for node, comm_id in partition.items():
        communities_by_id[comm_id].append(node)
    
    print(f"\n✅ Community assignments complete")

## Section 4: Greedy Modularity Communities

NetworkX's greedy modularity optimization for comparison.

In [ ]:
print("⏳ Running greedy modularity algorithm...")
from networkx.algorithms import community

greedy_communities = list(community.greedy_modularity_communities(G))
greedy_modularity = community.modularity(G, greedy_communities)

print(f"\n✅ Greedy Modularity Complete")
print(f"   Communities found: {len(greedy_communities)}")
print(f"   Modularity score: {greedy_modularity:.4f}")

# Show community sizes
greedy_sizes = sorted([(len(comm), i) for i, comm in enumerate(greedy_communities)], reverse=True)

print(f"\n📊 Community Sizes:")
for size, comm_id in greedy_sizes[:10]:
    print(f"   Community {comm_id}: {size} nodes")

## Section 5: Intra-Community and Inter-Community Edges

Analyze coupling within and between communities.

In [ ]:
if partition:
    # Calculate intra-community and inter-community edges
    intra_edges = 0  # Edges within communities
    inter_edges = 0  # Edges between communities
    
    for u, v in G.edges():
        if partition[u] == partition[v]:
            intra_edges += 1
        else:
            inter_edges += 1
    
    total_edges = G.number_of_edges()
    
    print("📊 Edge Distribution:")
    print(f"   Intra-community edges: {intra_edges} ({intra_edges/total_edges*100:.1f}%)")
    print(f"   Inter-community edges: {inter_edges} ({inter_edges/total_edges*100:.1f}%)")
    print(f"   Total edges: {total_edges}")
    
    print(f"\n💡 Insight:")
    print(f"   High intra-community edges = good modularity")
    print(f"   High inter-community edges = tightly coupled modules")
    
    # Visualize
    fig, ax = plt.subplots(figsize=(10, 6))
    sizes = [intra_edges, inter_edges]
    labels = [f'Intra-community\n({intra_edges})', f'Inter-community\n({inter_edges})']
    colors = ['#4F46E5', '#EC4899']
    ax.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', startangle=90, textprops={'fontsize': 12})
    ax.set_title('Edge Distribution: Intra vs Inter-Community', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()

## Section 6: Visualize Communities

Create network visualizations with communities highlighted.

In [ ]:
if partition:
    # Create figure
    plt.figure(figsize=(16, 12))
    
    # Get subgraph of largest component for visualization
    G_viz = G.subgraph(largest_cc).copy()
    
    # Layout
    pos = nx.spring_layout(G_viz, k=0.5, iterations=50, seed=42)
    
    # Create color map based on community
    num_communities = len(set(partition.values()))
    colors = plt.cm.tab20(np.linspace(0, 1, num_communities))
    
    # Draw edges
    nx.draw_networkx_edges(G_viz, pos, edge_color='gray', alpha=0.2, width=0.5)
    
    # Draw nodes colored by community
    for comm_id in set(partition.values()):
        nodes_in_comm = [node for node, c in partition.items() if c == comm_id and node in G_viz]
        if nodes_in_comm:
            nx.draw_networkx_nodes(G_viz, pos, nodelist=nodes_in_comm, 
                                   node_color=[colors[comm_id]], node_size=300, 
                                   label=f'Community {comm_id}')
    
    plt.title(f'Communities in Code Graph (Louvain, modularity={modularity:.3f})', 
             fontsize=16, fontweight='bold')
    plt.axis('off')
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=8)
    plt.tight_layout()
    plt.show()
    
    print("✅ Community visualization complete")

## Summary

This notebook demonstrated community detection for module identification:

1. ✅ **Connected Components** - Structural modularity
2. ✅ **Louvain Algorithm** - Optimal modularity optimization
3. ✅ **Greedy Modularity** - Fast community detection
4. ✅ **Edge Analysis** - Intra vs inter-community coupling
5. ✅ **Visualization** - Community-colored network graph

### Key Takeaways:
- **High modularity** (>0.4) indicates good separation of concerns
- **High inter-community edges** suggest tight coupling between modules
- **Communities align with architecture** → validation of design

### Next Step:
- **Notebook 04**: Architectural Patterns (detect smells)